In [1]:
# Import required packages (will fail if not available)
import xarray as xr
import pysnptools

print(f"✓ xarray {xr.__version__}")
print(f"✓ pysnptools {type(pysnptools)}")

✓ xarray 2025.7.1
✓ pysnptools <class 'module'>


In [2]:
# Get sample BED file using PySnpTools
from pysnptools.snpreader import Bed
from pysnptools.util import example_file

bed_file = example_file("tests/datasets/all_chr.maf0.001.N300.*")
snp_reader = Bed(bed_file, count_A1=True)
print(f"Shape: {snp_reader.shape} (individuals × SNPs)")

Shape: (300, 1015) (individuals × SNPs)


In [3]:
# Convert PySnpTools BED data to xarray Dataset with full genomic metadata
import numpy as np

# Create xarray Dataset using numeric indexes (avoids MultiIndex complexity and ensures Zarr compatibility)
xarray_form = xr.Dataset(
    {
        "genotypes": (["individual", "snp"], snp_reader.read().val)
    },
    coords={
        "individual": range(snp_reader.iid_count),
        "fid": (["individual"], [fid for fid, _iid in snp_reader.read().iid]),  # Family IDs
        "iid": (["individual"], [iid for _fid, iid in snp_reader.read().iid]),  # Individual IDs
        
        "snp": range(snp_reader.sid_count),
        "sid": (["snp"], snp_reader.sid),                                                 # SNP IDs
        "chromosome": (["snp"], np.nan_to_num(snp_reader.pos[:, 0], nan=0).astype(int)),  # Chromosome (NaN→0)
        "cm_position": (["snp"], snp_reader.pos[:, 1]),                                   # Genetic position
        "bp_position": (["snp"], np.nan_to_num(snp_reader.pos[:, 2], nan=0).astype(int)), # Physical position (NaN→0)
    },
    attrs={
        "description": "Genotype data from PySnpTools BED file",
        "encoding": "0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=missing",
        "source": bed_file,
    }
)

display(xarray_form)

<xarray.Dataset> Size: 3MB
Dimensions:      (individual: 300, snp: 1015)
Coordinates:
  * individual   (individual) int64 2kB 0 1 2 3 4 5 ... 294 295 296 297 298 299
    fid          (individual) <U4 5kB 'POP1' 'POP1' 'POP1' ... 'POP1' 'POP1'
    iid          (individual) <U4 5kB '0' '12' '44' ... '2987' '2998' '2999'
  * snp          (snp) int64 8kB 0 1 2 3 4 5 6 ... 1009 1010 1011 1012 1013 1014
    sid          (snp) <U10 41kB '1_12' '1_34' '1_10' ... '23_49' '23_2' '23_3'
    chromosome   (snp) int64 8kB 1 1 1 1 1 1 1 1 1 ... 23 23 23 23 23 23 23 23
    cm_position  (snp) float64 8kB 0.008008 0.02302 0.07007 ... 0.9479 0.995
    bp_position  (snp) int64 8kB 0 1 4 6 7 8 9 10 11 ... 44 46 47 48 49 50 51 52
Data variables:
    genotypes    (individual, snp) float64 2MB 0.0 0.0 1.0 0.0 ... 0.0 2.0 0.0
Attributes:
    description:  Genotype data from PySnpTools BED file
    encoding:     0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=mis...
    source:       C:\Users\carlk\AppData\Local\Temp/hashdown/072f91aaa72b5340...

In [4]:
# Import zarr explicitly and save to Zarr format
import zarr
import warnings
print(f"✓ zarr {zarr.__version__}")
warnings.filterwarnings('ignore', category=UserWarning, module='zarr') # Can ignore unicode warnings

# Save to Zarr format for efficient storage and access
zarr_path = "all_chr.maf0.001.N300.zarr"
xarray_form.to_zarr(zarr_path, mode='w')

# Verify by loading back lazily
zarr_form = xr.open_zarr(zarr_path)
display(zarr_form)


✓ zarr 3.0.10


<xarray.Dataset> Size: 2MB
Dimensions:      (individual: 300, snp: 1015)
Coordinates:
    chromosome   (snp) int64 8kB ...
    bp_position  (snp) int64 8kB ...
  * individual   (individual) int64 2kB 0 1 2 3 4 5 ... 294 295 296 297 298 299
    fid          (individual) object 2kB ...
    cm_position  (snp) float64 8kB ...
    iid          (individual) object 2kB ...
    sid          (snp) object 8kB ...
  * snp          (snp) int64 8kB 0 1 2 3 4 5 6 ... 1009 1010 1011 1012 1013 1014
Data variables:
    genotypes    (individual, snp) float64 2MB ...
Attributes:
    description:  Genotype data from PySnpTools BED file
    encoding:     0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=mis...
    source:       C:\Users\carlk\AppData\Local\Temp/hashdown/072f91aaa72b5340...

In [5]:
print(f"Dataset loaded with data variables: {list(zarr_form.data_vars.keys())}")
print(f"Dataset shape: {zarr_form.sizes}")
print(f"Coordinates preserved: {list(zarr_form.coords.keys())}")
print(f"Attributes preserved: {bool(zarr_form.attrs)}")

# Show that it works the same way
print(f"\nFirst individual from disk: ({zarr_form.fid.values[0]}, {zarr_form.iid.values[0]})")
print(f"First SNP from disk: {zarr_form.sid.values[0]}")
print(f"First genotype from disk: {zarr_form.genotypes.values[0, 0]}")

# Show efficient filtering examples with actual execution
print(f"\nFiltering examples:")

# By chromosome: get only chromosome 1 data
chr1_data = zarr_form.where(zarr_form.chromosome == 1, drop=True)
print(f"Chromosome 1 only: {chr1_data.sizes}")

# By family: get only POP1 family (though this dataset only has POP1)
pop1_data = zarr_form.where(zarr_form.fid == 'POP1', drop=True)
print(f"Family POP1 only: {pop1_data.sizes}")

# By SNP ID: get specific SNPs
specific_snps = zarr_form.where(zarr_form.sid.isin(['1_12', '1_34']), drop=True)
print(f"Specific SNPs: {specific_snps.sizes}")

# Combined filtering: chromosome 1 AND specific individuals
first_10_individuals = zarr_form.isel(individual=slice(0, 10))
chr1_subset = first_10_individuals.where(first_10_individuals.chromosome == 1, drop=True)
print(f"First 10 individuals, chromosome 1: {chr1_subset.sizes}")

Dataset loaded with data variables: ['genotypes']
Dataset shape: Frozen({'individual': 300, 'snp': 1015})
Coordinates preserved: ['chromosome', 'bp_position', 'individual', 'fid', 'cm_position', 'iid', 'sid', 'snp']
Attributes preserved: True

First individual from disk: (POP1, 0)
First SNP from disk: 1_12
First genotype from disk: 0.0

Filtering examples:
Chromosome 1 only: Frozen({'individual': 300, 'snp': 38})
Family POP1 only: Frozen({'individual': 300, 'snp': 1015})
Specific SNPs: Frozen({'individual': 300, 'snp': 2})
First 10 individuals, chromosome 1: Frozen({'individual': 10, 'snp': 38})


In [6]:
# read all genotype data from the zarr file
zarr_form.genotypes.values  # Accessing the genotype data directly

array([[0., 0., 1., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 2., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 2., 0.]], shape=(300, 1015))

In [14]:
# Read every second individual and SNPs (variants) from 20 to 30.
val2 = zarr_form.genotypes[::2, 20:30].values
val2.shape

(150, 10)

In [22]:
# List the first 5 individual (sample) ids, the first 5 SNP (variant) ids, and every unique chromosome. Then, read every value in chromosome 5.
print(zarr_form.iid[:5].values)
print(zarr_form.sid[:5].values)
print(np.unique(zarr_form.chromosome))
print(zarr_form.genotypes[:, zarr_form.chromosome == 5].values)



['0' '12' '44' '58' '65']
['1_12' '1_34' '1_10' '1_35' '1_28']
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[[0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
